In [167]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import math

In [168]:
#load data,channge type
load = np.genfromtxt("/Users/yueyuchen/Documents/ML/ML_Immune/Machine_Learning_Immunogenicity/data/bcell.csv",dtype = str, delimiter = ',')

length1 = np.shape(load)[0]
labels1 = np.zeros(length1)
load1 = np.zeros(length1)
for i in range(length1):
    labels1[i] = (load[i,1] == 'Positive')
for i in range(length1):   
    load1[i] = len(load[i,0])
data = []
labels = []
for i in range(length1):
    if(load1[i] < 20):
        data.append(load[i,0])
        labels.append(labels1[i])

length = len(data)


        
dataset = []
for d in data:
    d1 = []
    for l in d:
        d1.append(np.float32(ord(l)))
    for j in range(20-len(d1)):
        d1.append(np.float32(0))
    dataset.append(d1)

for i in range(len(labels)):
    if (labels[i] == 0):
        labels[i] = [1,0]
    else:
        labels[i] = [0,1]
        
for i in range(length):
    dataset[i] = np.asarray(dataset[i])
    labels[i] = np.float32(np.asarray(labels[i]))

dataset = np.asarray(dataset)
labels = np.asarray(labels)



In [205]:


shuf = np.append(dataset,labels,axis = 1)
np.random.shuffle(c)

dataset = c[:,:20]
labels = c[:,20:22]
print (np.shape(dataset),np.shape(labels))


(324231, 20) (324231, 2)


In [206]:

    
trainnum = 200000
testnum = 20000
train_dataset = dataset[:trainnum]
train_labels = labels[:trainnum]

test_dataset = dataset[trainnum:trainnum + testnum]
test_labels = labels[trainnum:trainnum + testnum]

valid_dataset = dataset[trainnum + testnum:trainnum + 2*testnum]
valid_labels = labels[trainnum + testnum:trainnum + 2*testnum]



In [207]:
print (dataset[5],)

[ 83.  71.  86.  70.  86.  78.  73.  78.  67.  67.  82.  68.  82.  83.  86.
   0.   0.   0.   0.   0.]


In [208]:


graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset)
  tf_train_labels = tf.constant(train_labels)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([20, 2]))
  biases = tf.Variable(tf.zeros([2]))
  
  # Training computation.
  
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [209]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
   
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels))
      
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 27.043997
Training accuracy: 78.9%
Validation accuracy: 78.7%
Loss at step 100: 13.752473
Training accuracy: 82.1%
Validation accuracy: 47.0%
Loss at step 200: 15.043095
Training accuracy: 66.6%
Validation accuracy: 80.4%
Loss at step 300: 89.045341
Training accuracy: 80.1%
Validation accuracy: 80.4%
Loss at step 400: 11.942018
Training accuracy: 82.6%
Validation accuracy: 79.0%
Loss at step 500: 11.944953
Training accuracy: 82.8%
Validation accuracy: 80.1%
Loss at step 600: 186.718765
Training accuracy: 80.1%
Validation accuracy: 80.4%
Loss at step 700: 11.746254
Training accuracy: 82.4%
Validation accuracy: 77.0%
Loss at step 800: 30.952021
Training accuracy: 80.2%
Validation accuracy: 80.1%
Test accuracy: 79.8%
